# Imports

In [ ]:
import numpy as np

In [ ]:

def spot_rates(prices, par_value, maturities, CF):
    """
    Calculate spot rates for different maturities using zero-coupon bond prices.

    :param prices: A list of zero-coupon bond prices.
    :param par_value: The face value (par value) of the bond.
    :param maturities: A list of maturities for the bonds in years.
    :CF: Compounding Frequency
    :return: A list of spot rates for different maturities.
    """
    spot_rates = []
    for i in range(len(prices)):
        pv = prices[i]
        t = maturities[i]*CF
        spot_rate = ((par_value[i] / pv) ** (1. / t) )- 1

        spot_rates.append(spot_rate*CF)
    return spot_rates

def forward_rates(spot_rates, maturities, CF):
    """
    Calculate forward rates from spot rates for different maturities.

    :param spot_rates: A list of spot rates.
    :param maturities: A list of maturities for the forward rates in years.
    :CF: Compounding Frequency
    :return: A list of forward rates.
    """
    forward_rates = []
    for i in range(len(spot_rates) - 1):
        sr_t1 = spot_rates[i] / CF
        sr_t2 = spot_rates[i + 1] / CF
        t1 = maturities[i]*CF
        t2 = maturities[i + 1]*CF
        forward_rate = ((1 + sr_t2) ** t2 / (1 + sr_t1) ** t1) ** (1 / (t2 - t1)) - 1
        forward_rates.append(forward_rate*CF)
    return forward_rates

# Example usage:
bond_prices = [99, 98]  # Prices of zero-coupon bonds
face_value = [100,100]  # Face value of the bonds
bond_maturities = [0.5, 1]  # Maturities of the bonds in years
comp_freq = 2
spot_rates_result = spot_rates(bond_prices, face_value, bond_maturities,comp_freq)
print("Spot Rates:", spot_rates_result)

forward_rates_result = forward_rates(spot_rates_result, bond_maturities,comp_freq)
print("Forward Rates:", forward_rates_result)


# Example usage:
bond_prices = [980, 950, 920, 900, 880]  # Prices of zero-coupon bonds
face_value = [1000,1000,1000,1000,1000]  # Face value of the bonds
bond_maturities = [1, 2, 3, 4, 5]  # Maturities of the bonds in years
comp_freq = 1
spot_rates_result = spot_rates(bond_prices, face_value, bond_maturities,comp_freq)
print("Spot Rates:", spot_rates_result)

forward_rates_result = forward_rates(spot_rates_result, bond_maturities,comp_freq)
print("Forward Rates:", forward_rates_result)

1.0101010101010102
1.0204081632653061
Spot Rates: [0.020202020202020332, 0.020305089104421636]
Forward Rates: [0.020408163265306367]
1.0204081632653061
1.0526315789473684
1.0869565217391304
1.1111111111111112
1.1363636363636365
Spot Rates: [0.020408163265306145, 0.025978352085153977, 0.02818372270192615, 0.026690096080340897, 0.02589630491023409]
Forward Rates: [0.03157894736842071, 0.03260869565217428, 0.0222222222222217, 0.02272727272727315]


In [ ]:
# def present_value(cash_flows,spot_rates,maturities,CF):
#   pv = 0
#   for i in range(len(cash_flows)):
#     sr = spot_rates[i]/CF
#     t = maturities[i]*CF
#     pv += (cash_flows[i])/(1+sr)**t

#   return pv


def present_value(cashflows, Maturities, spot_rates, CF):
    """
    Calculate the present value of a list of cashflows using a given rate.

    :param cashflows: A list of cashflows, including coupon payments and face value.
    :param rate: The discount rate for calculating the present value.
    :return: The present value of the cashflows.
    """
    pv = 0
    for i in range(len(cashflows)):
        pv += cashflows[i] / (1 + spot_rates[Maturities[i]*CF]) ** (Maturities[i]*CF)
    return pv

def spot_rates_nz(prices, cash_flows, maturities, CF):
  spot_rates = []
  spot_rates.append((cash_flows[0][0]/prices[0] - 1) * CF)

  for i in range(1,len(prices)):
    print(cash_flows[i])
    a_pv = present_value(cash_flows[i][:i],spot_rates,maturities[i],CF)
    pv = prices[i]  - a_pv
    print(pv)
    t = maturities[i][i]*CF
    print(cash_flows[i][i])
    spot_rates.append(((cash_flows[i][i]/pv)** (1/t)- 1) * CF)

  return spot_rates


def spot_rates_bond(PV, CR, FV, CF, N):
  cash_flows = []
  maturities = []

  for i in range(len(PV)):
    cash_flow = []
    maturitie = []
    for j in range(int(CF*N[i])-1):
      cash_flow.append(FV[i]*CR[i]/CF)
      maturitie.append((j+1)/CF)
    cash_flow.append(FV[i]*(1+(CR[i]/CF)))
    cash_flows.append(cash_flow)
    maturitie.append(N[i])
    maturities.append(maturitie)

  spot_rates = spot_rates_nz(PV,cash_flows,maturities,CF)

  return spot_rates

PV = [99,98]
CR = [0.06,0.06]
FV = [100,100]
CF = 2
N = [0.5,1]

spot_rates = spot_rates_bond(PV,CR,FV,CF,N)
spot_rates

[3.0, 103.0]
95.11650485436893
103.0


[0.08080808080808088, 0.08123282220479044]

In [ ]:


def calculate_irr(cashflows, spot_rates, Maturities, tolerance=1e-6, max_iterations=1000):
    """
    Calculate the Internal Rate of Return (IRR) using cashflows and spot rates.

    :param cashflows: A list of cashflows.
    :param spot_rates: A list of spot rates corresponding to the cashflow periods.
    :param tolerance: The desired precision for the IRR calculation.
    :param max_iterations: The maximum number of iterations for the IRR calculation.
    :return: The calculated IRR.
    """
    guess = 0.1  # Initial guess for IRR
    for _ in range(max_iterations):
        present_value = present_value(cashflows, Maturities, SR, CF)
        deriv_present_value = calculate_present_value([c * (i + 1) for i, c in enumerate(cashflows)], spot_rates)
        new_guess = guess - present_value / deriv_present_value
        if abs(new_guess - guess) < tolerance:
            return new_guess
        guess = new_guess
    raise ValueError("IRR calculation did not converge within the given number of iterations.")


def macaulay_duration(cashflows, Maturitites, SR, CF):
    """
    Calculate the Macaulay Duration of a bond using the given cashflows and yield.

    :param cashflows: A list of cashflows, including coupon payments and face value.
    :param rate: The yield of the bond.
    :return: The Macaulay Duration of the bond.
    """
    pv_total = present_value(cashflows, Maturitites, SR, CF)
    macaulay_duration = 0
    for i in range(len(cashflows)):
      macaulay_duration += (cashflows[i]/ (1 + SR[Maturities[i]*CF]) ** (Maturities[i]*CF)) * Maturities[i]

    return macaulay_duration/pv_total

def modified_duration(cashflows, Maturities, SR, CF):
    """
    Calculate the Modified Duration of a bond using the given cashflows and yield.

    :param cashflows: A list of cashflows, including coupon payments and face value.
    :param rate: The yield of the bond.
    :return: The Modified Duration of the bond.
    """
    macaulay_dur = macaulay_duration(cashflows, Maturities, SR, CF)
    IRR = calculate_irr(cashflows, spot_rates, Maturities)
    modified_duration = macaulay_dur / (1 + IRR/CF)
    return modified_duration

def effective_duration(cashflows, Maturities, SR, CF, delta_rate=0.01):
    """
    Calculate the Effective Duration of a bond using the given cashflows and yield, with a small change in yield.

    :param cashflows: A list of cashflows, including coupon payments and face value.
    :param rate: The yield of the bond.
    :param delta_rate: A small change in yield used to calculate the effective duration.
    :return: The Effective Duration of the bond.
    """
    pv = present_value(cashflows, Maturities, SR, CF)

    IRR = calculate_irr(cashflows, SR, Maturities)
    pv_minus = present_value(cashflows, Maturities, [IRR-delta_rate]*len(SR))
    pv_plus = present_value(cashflows, Maturities, [IRR+delta_rate]*len(SR))
    effective_duration = (pv_minus - pv_plus) / (2 * delta_rate * pv)
    return effective_duration

# Example usage:
bond_cashflows = [50, 50, 50, 50, 50, 1050]  # Cashflows including coupon payments and face value
bond_yield_rate = 0.05  # Yield to maturity (YTM) of the bond
bond_price = 1050  # Price of the bond

macaulay_duration_result = macaulay_duration(bond_cashflows, bond_yield_rate)
print("Macaulay Duration:", macaulay_duration_result)

modified_duration_result = modified_duration(bond_cashflows, bond_yield_rate)
print("Modified Duration:", modified_duration_result)

effective_duration_result = effective_duration(bond_cashflows, bond_yield_rate)
print("Effective Duration:", effective_duration_result)


TypeError: ignored

In [ ]:
def compute_conversion_factor(coupon_rate, yield_rate, time_to_maturity, coupon_frequency):
    """
    Compute the Conversion Factor for a Treasury bond futures contract.

    :param coupon_rate: The annual coupon rate of the underlying bond.
    :param yield_rate: The yield of the underlying bond as a decimal (e.g., 0.05 for 5% yield).
    :param time_to_maturity: Time to maturity of the bond in years until the futures delivery date.
    :param coupon_frequency: The number of coupon payments per year (e.g., 2 for semi-annual coupons).
    :return: The Conversion Factor.
    """
    n = coupon_frequency * time_to_maturity
    coupon_payment = (coupon_rate / coupon_frequency) * 100  # Coupon payment per period based on $100 face value
    discount_factor = 1 / (1 + yield_rate / coupon_frequency) ** n

    present_value_cash_flows = coupon_payment * (1 - discount_factor) / (yield_rate / coupon_frequency)
    conversion_factor = present_value_cash_flows / (100 * discount_factor)
    return conversion_factor

# Example usage:
coupon_rate = 0.05  # Annual coupon rate (e.g., 5%)
yield_rate = 0.04  # Yield of the underlying bond (e.g., 4% yield)
time_to_maturity = 3  # Time to maturity in years until the futures delivery date
coupon_frequency = 2  # Number of coupon payments per year (e.g., 2 for semi-annual coupons)

conversion_factor = compute_conversion_factor(coupon_rate, yield_rate, time_to_maturity, coupon_frequency)
print("Conversion Factor:", conversion_factor)


In [ ]:
def find_cheapest_to_deliver_bond(futures_price, conversion_factors, cash_prices):
    """
    Find the Cheapest to Deliver (CTD) bond from a basket of eligible bonds.

    :param futures_price: The quoted price of the futures contract (per $100 face value).
    :param conversion_factors: A list of Conversion Factors for the eligible bonds.
    :param cash_prices: A list of cash prices for the eligible bonds.
    :return: The index of the CTD bond in the list of eligible bonds.
    """
    cheapest_delivery_cost = float('inf')
    ctd_bond_index = -1

    for i, conversion_factor in enumerate(conversion_factors):
        delivery_cost = futures_price * conversion_factor / cash_prices[i]
        if delivery_cost < cheapest_delivery_cost:
            cheapest_delivery_cost = delivery_cost
            ctd_bond_index = i

    return ctd_bond_index

# Example usage:
futures_price = 101.50  # Quoted price of the futures contract per $100 face value
conversion_factors = [1.05, 1.06, 1.03]  # Conversion Factors for eligible bonds
cash_prices = [1000, 1020, 980]  # Cash prices for eligible bonds

ctd_bond_index = find_cheapest_to_deliver_bond(futures_price, conversion_factors, cash_prices)
print("Cheapest to Deliver Bond Index:", ctd_bond_index)
